# Motor Encoder 
**Full Coding At [Motor_Encoder.py](Motor_Encoder.py)** <br>
- *Library Used* 
    - RPi_GPIO   
    - adafruit_ssd1306  
    - board  
    - time 

## Installing Libraries (Dependencies)
- adafruit_ssd1306 
    - `pip3 install adafruit-circuitpython-ssd1306 `
- RPi.GPIO 
    - `pip install rpi-lgpio` 
- font5x8.bin *(This File is required as it is the font file for the OLED Diaplay to display Text)*
    - `wget https://github.com/adafruit/Adafruit_CircuitPython_framebuf/raw/main/examples/font5x8.bin`

## Let's Start Coding ! 
### 1. Import all the libraries 
- RPi.GPIO 
- adafruit_ssd1306 
- board
- time 

In [ ]:
import RPi.GPIO as GPIO 
import adafruit_ssd1306
import board 
import time


### 2. Create the Encoder Class
**As this Script will be a Library for other script thus a class is needed* 
- `isinit` <br> 
    Is used to make sure the library is not being initialise multiple time 

In [ ]:

class Encoder:
    """
    Class to handle encoder inputs and display RPM values on an OLED display.
    """
    isinit = False

### 3. The ` __init__` function. 
- The `__init__` function will run when the code is initialise 
- In default the debug flag is set to false. 
- `ENCODER_RES` is the resolution(ppr) of the motor encoder. In default is 13ppr 
- ` Gear Ratio` is set according to the motor rating (1:30) 
- `LEFT_HALLSEN=11` is the input pin of the left sensor(11)
- `RIGHT_HALLSEN=10` is the input of the right sensor(10)
- The  `ODISPLAY` is represent the Oled DIsplay. It is set to False in default 
- The `OLED_addr` is used to defined the i2c address of the oled screen. In default is 0x3c 
- If the *debug* flag is `True`, then the script will display more function. 
- In the script, only if the ODISPLAY is `True`, the oled library will be initialise.
- This function will check is the Library being initialised before. To prevent error due to double initialisation 

In [ ]:
    def __init__(self, debug=False, ENCODER_RES=13, gear_ratio=30, LEFT_HALLSEN=11, RIGHT_HALLSEN=10, ODISPLAY=False, OLED_addr=0x3c):
        """
        Initialize the Encoder object with specified parameters.
        """
        if not Encoder.isinit:
            self.debug = debug 
            self.ODISPLAY = ODISPLAY 
            self.ENCODER_RES = ENCODER_RES 
            self.LEFT_HALLSEN = LEFT_HALLSEN
            self.RIGHT_HALLSEN = RIGHT_HALLSEN
            self.gear_ratio = gear_ratio
            self.OLED_addr = OLED_addr 
            
            self.left_enc_val = 0
            self.right_enc_val = 0
            self.i2c = board.I2C()
            self.oled = adafruit_ssd1306.SSD1306_I2C(128, 64, self.i2c, addr=OLED_addr)
            
            if self.debug:
                print("Encoder Resolution =", self.ENCODER_RES)
                print("Gear Ratio =", self.gear_ratio)
                print("Left Hall Sensor Pin =", self.LEFT_HALLSEN)
                print("Right Hall Sensor Pin =", self.RIGHT_HALLSEN)
                print("OLED Display Enabled =", self.ODISPLAY)
                print("OLED address =", self.OLED_addr)
                print("Debug Mode Enabled =", self.debug)
                print("I2C Initialization Complete -- Motor_Encoder")
                print("All values initialized -- Motor_Encoder")
                
            if self.ODISPLAY:
                self.i2c = board.I2C()
                self.oled = adafruit_ssd1306.SSD1306_I2C(128, 64, self.i2c, addr=OLED_addr)
                print("OLED Display Enabled")
                self.oled.fill(1)
                self.oled.show()
                self.oled.fill(0)
                self.oled.text("OLED ENABLED", 25, 30, 1)
                self.oled.show()
                time.sleep(1)
            Encoder.isinit = True 

### 4. The `setup` function 
- This code will **setup all the GPIO pins** to initialise the encoder. 

- When the function start it will clean all the command to the GPIO pins 
    ```
    GPIO.cleanup()
    ``` 
    - This is to ensure there is no pin conflict when running the script 

- The Raspberry Pi's General Purpose Input/Output (GPIO) pins can be accessed in two different modes:

    - Board Mode: This mode uses the physical pin numbers on the Raspberry Pi board. For example, pin 1 refers to the first pin on the board, pin 2 refers to the second pin, and so on.

    - BCM Mode: This mode uses the Broadcom (BCM) chip-specific pin numbers. These numbers refer to the actual GPIO pin numbers defined by the Broadcom SOC (System on Chip).

- In this function the GPIO pin are set to the BCM mode. 
- It also set the pin for the sensor to input mode 
- To detect the change of the pin signal (from 1 to 0 and 0 to 1) event detection is added. 
- This process will keep repeating until the edge detection is successfully added to the pins.
- If the **debug** flag is **True**, The function will print the GPIO mode and also the repeat count. 
- When the event detection is failed to add for 3 times. It will prompt an error and stop te script. 
- If all the setup above is sucess, the OLED Diaplay will show 
    > Motor Encoder <br>
    > Setup Complete 


In [ ]:
    def setup(self):
        """
        Set up GPIO pins and initialize encoder.
        """
        retry_count = 3
        while retry_count > 0:
            try:
                GPIO.cleanup() # Clean up GPIO
                GPIO.setmode(GPIO.BCM) # Set GPIO Mode to BCM
                GPIO.setwarnings(False) 
                GPIO.setup(self.LEFT_HALLSEN, GPIO.IN)
                GPIO.setup(self.RIGHT_HALLSEN, GPIO.IN) 
                if self.debug:
                    current_Mode = GPIO.getmode()
                    print("Current GPIO Mode:", current_Mode)
                    print(f"Setting up edge detection for pins: LEFT={self.LEFT_HALLSEN}, RIGHT={self.RIGHT_HALLSEN}")
                GPIO.add_event_detect(self.LEFT_HALLSEN, GPIO.BOTH, callback=self.left_update)
                GPIO.add_event_detect(self.RIGHT_HALLSEN, GPIO.BOTH, callback=self.right_update)
                if self.debug:
                    print("GPIO edge detection added successfully.")
                break
            except RuntimeError as e:
                retry_count -= 1
                print(f"Error adding edge detection: {e}. Retries left: {retry_count}")
                time.sleep(1)
                GPIO.remove_event_detect(self.LEFT_HALLSEN)
                GPIO.remove_event_detect(self.RIGHT_HALLSEN)
                GPIO.cleanup()  # Ensure GPIO cleanup before retrying

        if retry_count == 0:
            print("Failed to add edge detection after multiple attempts.")
            raise RuntimeError("Failed to add edge detection after multiple attempts.")

        if self.debug:
            print("GPIO Initialization Complete -- Motor_Encoder")
            
        self.left_enc_val = 0
        self.right_enc_val = 0 
        
        if self.ODISPLAY:
            self.oled.fill(0)
            self.oled.show()
            self.oled.text("MOTOR ENCODER", 23, 20, 1)
            self.oled.text("Setup Complete", 20, 35, 1)
            self.oled.show()
            time.sleep(1)

## 5. The `encoder` function 
- This code is used to record and calculate the RPM of the wheels based on the **Gear Ratio** and **Resolution** of the encoder 
- The calculation of RPM involved time start and time end thus the time library is used to obtained the time.
    ```
    start_time = time.time()
    ``` 
- The start value (Maybe 0 or 1 depense on the hall effect sensor) is saved in to the left_start_enc_val and right_start_enc_val variables. 

- After some time of delay (In default is 0.1 second), it will start to obtained the left_end_enc_val and right_end_enc_val in to variables and also the end time. 

- The pulse count can be calculated by subtracting the end value to the start value (right_end_enc_val - right_start_enc_val). 

- Other than the pulse count, another paramerter is needed which is the time interval. The tiume interval is obtained by subtracting the end time with the start time. (end_time - satrt_time). 

- After havimg this two parameters (pulse_count and time interval) we can calculate the RPM of the car using the Formula: 
    $$
     RPM = (Pulse Count /Encoder Resolution(PPR)) * (60/Time Interveral) ​* Gear Ratio
    $$
    in this script the formula is written as: 
    ```py
    right_rpm = (right_pulse_count * 60) / (time_interval * self.ENCODER_RES) / self.gear_ratio
    ```
- This function will return the value of left and right rpm  calculated. 
- If the Oled Diaplay (ODISPLAY) flag is **True**, This function will also used to diaplay the value of calculated RPM on the OLED Display. 



In [ ]:
 def encoder(self, readback_duration=0.1):
        """
        Read encoder values and calculate RPM.
        """
        
        start_time = time.time()
        left_start_enc_val = self.left_enc_val
        right_start_enc_val = self.right_enc_val

        time.sleep(readback_duration) 
        
        left_end_enc_val = self.left_enc_val 
        right_end_enc_val = self.right_enc_val 
        
        end_time = time.time()
        
        left_pulse_count = left_end_enc_val - left_start_enc_val
        right_pulse_count = right_end_enc_val - right_start_enc_val
        
        time_interval = end_time - start_time
        
        left_rpm = (left_pulse_count * 60) / (time_interval * self.ENCODER_RES) / self.gear_ratio
        right_rpm = (right_pulse_count * 60) / (time_interval * self.ENCODER_RES) / self.gear_ratio

        if self.ODISPLAY:
            self.oled.fill(0)
            self.oled.text("Left Motor:", 1, 10, 1)
            self.oled.text("{:.2f} rpm".format(left_rpm), 1, 25, 1)
            self.oled.text("Right Motor:", 1, 40, 1)
            self.oled.text("{:.2f} rpm".format(right_rpm), 1, 55, 1)
            self.oled.show()
        
        return left_rpm, right_rpm

## 6. The `left_update` and `right_update` functions 
- This function is use to add the pulse count when the event_detector detecteed some change from the GPIO. 
- When the value of the encoder switch from (0 to 1 or 1 to 0) it will trigger this function to add the pulse count. 


In [ ]:
    def left_update(self, channel): 
        """
        Callback function for left encoder interrupt.
        """
        self.left_enc_val += 1
    
    def right_update(self, channel): 
        """
        Callback function for right encoder interrupt.
        """
        self.right_enc_val += 1 

## 7. The `stop` function 
- This function is called at the last of the code to stop all the process above. 
- This will clear the GPIO Configuration using 
    ```
    GPIO.cleanup()
    ``` 
- If the OLED Display (ODIAPLAY) flag is **True** this will also clear the Oled display. 


In [ ]:
    def stop(self):
        """
        Clean up GPIO resources and OLED display.
        """
        GPIO.cleanup() 
        if self.ODISPLAY:
            self.oled.fill(0)
            self.oled.show()
            self.i2c.deinit()

## 8.The `if` statement 
- As this script will also be used as a customised library, when this script is run alone (Without importing toi the other script) the `__name__` of this script wil be come `__main__` which means the main script run in python 
- This will start the encoder without starting the motor.
- To get value displayed on the OLED screen, kindly turn the Motor. 
- When the motor turned, the RPM value of the Motor kan be viewed on the terminal and also the OLED Display. 
- When a keyboard inturuption *(Ctrl + C)* press is detected this will call the script to stop and exit.

In [ ]:
if __name__ == '__main__':
    enc = Encoder(ODISPLAY=True, debug=True)
    try: 
        while True:
            left_rpm, right_rpm = enc.encoder()
            print("Left Motor: {:.2f}".format(left_rpm))  
            print("Right Motor: {:.2f}".format(right_rpm))
            time.sleep(1)
    except KeyboardInterrupt:
        enc.stop()